In [1]:
import infomap

In [2]:
#import uniout
import numpy as np
from gensim.models import Word2Vec
from tqdm import tqdm
#from infomap import infomap # v0.x
import infomap # v1.0

num_words = 10000 # 只保留前10000个词
min_sim = 0.6


word2vec = Word2Vec.load('baike_word2vec/word2vec_baike')

word_vecs = word2vec.wv.syn0[:num_words]
word_vecs /= (word_vecs**2).sum(axis=1, keepdims=True)**0.5
id2word = word2vec.wv.index2word[:num_words]
word2id = {j: i for i, j in enumerate(id2word)}

# 构造[wordA，wordB，相似性]
links = {}

# 每个词找与它相似度不小于0.6的词（不超过50个），来作为图上的边
for i in tqdm(range(num_words)):
    sims = np.dot(word_vecs, word_vecs[i])
    idxs = sims.argsort()[::-1][1:]
    for j in idxs[:50]:
        if sims[j] >= min_sim:
            links[(i, j)] = float(sims[j])
        else:
            break


# 方式一(infomap模型初始化)：Infomap直接addLink

infomapWrapper = infomap.Infomap("--two-level --directed")
#infomapWrapper = infomap.Infomap("--two-level")
# 如果重叠社区发现，则只需要：
# infomapWrapper = infomap.Infomap("--two-level --directed --overlapping")

for (i, j), sim in tqdm(links.items()):
    #print(i, j,sim)
    _ = infomapWrapper.addLink(int(i), int(j),sim)

# 方式二(infomap模型初始化)：network 添加addLink
infomapWrapper = infomap.Infomap("--two-level --directed")
network = infomapWrapper.network() 
for (i, j), sim in tqdm(links.items()):
    network.addLink(int(i), int(j),sim)

# 聚类运算
infomapWrapper.run()

# 有多少聚类数
print("Found {} modules with codelength: {}".format(infomapWrapper.numTopModules(), infomapWrapper.codelength()))

# 聚类结果显示
word2class = {}
class2word = {}
# for node in tree.leafIter():
#     if id2word[node.physIndex] not in word2class:
#         word2class[id2word[node.physIndex]] = []
#     word2class[id2word[node.physIndex]].append(node.moduleIndex())

#     if node.moduleIndex() not in class2word:
#         class2word[node.moduleIndex()] = []
#     class2word[node.moduleIndex()].append(id2word[node.physIndex])
for node in tree.iterTree():
    if id2word[node.physicalId] not in word2class:
        word2class[id2word[node.physicalId]] = []  # node.physicalId 词的编号
    word2class[id2word[node.physicalId]].append(node.moduleIndex())  # node.moduleIndex() 聚类的编号

    if node.moduleIndex() not in class2word:
        class2word[node.moduleIndex()] = []
    class2word[node.moduleIndex()].append(id2word[node.physicalId])


for i in range(100):
    print('---------------')
    print (class2word[i][1:])




FileNotFoundError: [Errno 2] No such file or directory: 'baike_word2vec/word2vec_baike'

文件找不到的解决方案
https://blog.csdn.net/a1368783069/article/details/52025764